In [ ]:
!pip install sentence-transformers transformers PyMuPDF python-docx

In [ ]:
from docx import Document  # python-docxを利用してWordを操作
import fitz  # PyMuPDFを利用してPDFを操作
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# ドキュメントを読み込み、テキストを抽出する関数
def load_pdf(file_path):
    doc = fitz.open(file_path)
    text_data = []
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text_data.append(page.get_text())
    return text_data

# PDFを読み込み、前処理を行う
pdf_text = load_pdf('/content/2450-900-02E.pdf')

FileNotFoundError: no such file: '/2450-900-02E.pdf'

In [ ]:
# Wordファイルを読み込み、テキストを抽出する関数
def load_docx(file_path):
    doc = Document(file_path)
    text_data = []
    for paragraph in doc.paragraphs:
        text_data.append(paragraph.text)
    return text_data

# Wordファイルを読み込み、前処理を行う
docx_text = load_docx('/content/2450-900-02E.docx')  # ここにWordファイルのパスを指定


In [ ]:
# SentenceTransformerのモデルをロードして、検索機能を設定
retrieval_model = SentenceTransformer('all-MiniLM-L6-v2')

# PDFまたはDOCXの各ページを埋め込み（ベクトル）としてエンコード
#page_embeddings = retrieval_model.encode(pdf_text, convert_to_tensor=True)
page_embeddings = retrieval_model.encode(docx_text, convert_to_tensor=True)

In [ ]:
# 日本語のGPT-2モデルをロード
model_name = "rinna/japanese-gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/806k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# RAGシステムの定義
def rag_retrieve_and_generate(query, num_retrievals=3, max_input_length=512):
    # クエリをエンコードして、関連するドキュメントを取得
    query_embedding = retrieval_model.encode(query, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, page_embeddings)[0]
    top_k_idx = scores.topk(num_retrievals).indices

    # 上位の取得結果をまとめてコンテキストを生成
    #context = "\n".join([pdf_text[idx] for idx in top_k_idx])
    context = "\n".join([docx_text[idx] for idx in top_k_idx])
    # 入力テキストにクエリとコンテキストを結合し、最大長を制限
    input_text = f"コンテキスト: {context}\n質問: {query}\n答え:"
    input_text = input_text[:max_input_length]  # 入力を最大長で切り捨て

    # トークナイズ（`attention_mask`と`pad_token_id`を指定）
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=max_input_length, truncation=True)
    attention_mask = tokenizer.encode_plus(input_text, max_length=max_input_length, truncation=True, return_tensors="pt")["attention_mask"]

    # テキスト生成（`max_new_tokens`で生成トークン数を指定）
    output = model.generate(inputs, max_new_tokens=50, do_sample=True, top_p=0.95, top_k=50, attention_mask=attention_mask, pad_token_id=tokenizer.eos_token_id)

    # 生成されたテキストをデコード
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text


In [ ]:
# RAGシステムをテストする
query = "Model 2450 SourceMeterの主な特徴は何ですか？"
answer = rag_retrieve_and_generate(query)
print(answer)

コンテキスト: デジタル入出力コネクタ・ピンについての詳細は、『odel 2450 eference anual』の「igital eference anual』の「2450 in a odel 2400 application」を参照してください。 eference anual』の「2450 in a odel 2400 application」を参照してください。 質問: odel 2450 ourceeterの主な特徴は何ですか? 答え:odel 2450 ourceeter は、デジタル入出力コネクタ・ピンセットを使用するために必要なすべての信号を提供するため、標準のコンフィギュレーションと追加のオプションを追加し、コネクタ・ピンセット・インターフェースを


In [ ]:
def filter_context(text):
    # より厳密に「特徴」「概要」などの情報をフィルタリング
    keywords = ["特徴", "機能", "仕様", "概要", "性能"]
    filtered_lines = [line for line in text.splitlines() if any(keyword in line for keyword in keywords)]
    return "\n".join(filtered_lines) if filtered_lines else text

def rag_retrieve_and_generate(query, num_retrievals=2, max_input_length=250):
    # クエリをエンコードして、関連するドキュメントを取得
    query_embedding = retrieval_model.encode(query, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, page_embeddings)[0]
    top_k_idx = scores.topk(num_retrievals).indices

    # フィルタリング後のコンテキストのみを取得
    #context = "\n".join([pdf_text[idx] for idx in top_k_idx])
    context = "\n".join([docx_text[idx] for idx in top_k_idx])
    context = filter_context(context)[:max_input_length]

    # モデルに渡すプロンプトをシンプル化
    input_text = f"2450型ソースメータの主な特徴:\n{context}\n質問: {query}\n簡潔に答えてください。"
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=max_input_length, truncation=True)
    attention_mask = tokenizer.encode_plus(input_text, max_length=max_input_length, truncation=True, return_tensors="pt")["attention_mask"]

    # 生成のトークン数を抑制（max_new_tokens=50）
    output = model.generate(inputs, max_new_tokens=50, do_sample=True, top_p=0.9, top_k=50, attention_mask=attention_mask, pad_token_id=tokenizer.eos_token_id)

    # 生成されたテキストをデコード
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text



2450型ソースメータの主な特徴: デジタル入出力コネクタ・ピンについての詳細は、『odel 2450 eference anual』の「igital eference anual』の「2450 in a odel 2400 application」を参照してください。 質問: odel 2450 ourceeterは何のための装置ですか? 簡潔に答えてください。注: odel 2450 ourceeterは次の機能を提供します: 2450 pro  electron editor 2450 pro    


In [ ]:
# テスト実行
#query = "Model 2450 SourceMeterの主な特徴は何ですか？"
query = "Model 2450 SourceMeterは何のための装置ですか？"
answer = rag_retrieve_and_generate(query)
print(answer)

In [ ]:
!pip install openai==0.28.0

In [ ]:
import openai

# OpenAI APIキーを設定（事前に取得しておく必要があります）# セキュリティのため非公開
openai.api_key = ""  # セキュリティのため非公開

# コンテキストとクエリをgpt-4-turboに渡して応答を生成する関数
def rag_retrieve_and_generate_gpt4(query, num_retrievals=2, max_input_length=250):
    # クエリをエンコードして、関連するドキュメントを取得
    query_embedding = retrieval_model.encode(query, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, page_embeddings)[0]
    top_k_idx = scores.topk(num_retrievals).indices

    # フィルタリング後のコンテキストのみを取得
    context = "\n".join([docx_text[idx] for idx in top_k_idx])
    #context = filter_context(context)[:max_input_length]

    # OpenAI APIでの応答生成用のプロンプトを設定
    prompt = f"コンテキスト: {context}\n質問: {query}\n簡潔に答えてください。"

    # OpenAIのChatCompletion APIを呼び出し
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini-2024-07-18",  # ここで指定するモデルIDを変更
        messages=[
            {"role": "system", "content": "あなたはユーザーの質問に的確に答えるアシスタントです。"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        temperature=0.7,
        top_p=0.9
    )

    # 応答のテキスト部分を取り出し
    generated_text = response['choices'][0]['message']['content']
    return generated_text


In [ ]:
# テスト実行
#query = "Model 2450 SourceMeterの主な特徴は何ですか？"
query = "Model 2450 SourceMeterは何のための装置ですか？"
answer = rag_retrieve_and_generate_gpt4(query)
print(answer)

Model 2450 SourceMeterは、電流や電圧を供給し、同時に測定するための計測器で、主に電子部品や材料の特性評価に使用されます。


In [ ]:
この装置の概要を出力したり、みんなにわかる回答を生成するための質問集を作ってくる。

SyntaxError: invalid character '、' (U+3001) (<ipython-input-26-762ae27e4fbc>, line 1)

In [ ]:
画像への対処、モデルの精査、やっぱりpdfはうまくいかない？？
pdfをそのまま読み込んだ場合と、wordで再保存してから読み込んだ場合の比較